In [4]:
import re
import requests
import csv
import time

from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

pd.options.display.float_format = '{:.3f}'.format

In [ ]:
## Scrape historical PGA data

## Functionally, this is what I am trying to do:
#1) Open up each tournament going back to 2012 (Fortinet, this particular tourney, just a start)
#2) Once the tournament HTML has loaded, "click" on each players name, so that hole-by-hole data is available
#3) Once you have "clicked" on a player's name, you are required to do a few more clicks such that the HTML becomes available:
#    a) click through all of rounds 1-4, so that you have all rounds
#    b) click the "arrow" button (at the top right of the players hole-by-hole scores) to toggle between the 
#    front 9 holes and back 9 holes.
#4) The "issue" I am having is that selenium seems to only find the "round" buttons (rounds 1,2,3,4 within the players explanded scorecard)
#   if the devtools pane is open on the page being controlled by the webdriver. I stumbled upon this literally by accident (I have
#   to manually open devtools on the driver page as my selenium code runs in order for selenium to "find" the round buttons ...)
#   No idea what would cause this but very curious to understand why this behavior is happening.


#### From pga_scrape.py

In [27]:
driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 101.0.4951
[WDM] - Get LATEST chromedriver version for 101.0.4951 google-chrome
[WDM] - Driver [C:\Users\phil0\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache
C:\Users\phil0\AppData\Local\Temp\ipykernel_26036\1503906442.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [36]:
print(len(''))

0


In [39]:
driver.get("https://www.pgatour.com/competition/2022/fortinet-championship/leaderboard.html")
driver.maximize_window()

#These are div elements which when you "click" on them, they expand to show a player's full hole-by-hole scoring, which is what we are after
#The hole by hole HTML seems to not be present in the DOM until you click on the name and expand the data (I think)

name_elements = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "player-name-col")))
# name_elements = driver.find_elements_by_class_name('player-name-col')

num_players = len(name_elements)

for i in range(num_players):
    player_ele = name_elements[i]
    player_name = player_ele.text
    player_ele.click()
    score_elems, scores = [], []
    score_elems.extend(WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'score-wrapper'))))
    icon_backward = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, 'icon-backward')))
    icon_backward.click()
    score_elems.extend(WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'score-wrapper'))))
    print(len(score_elems))
    for score_elem in score_elems:
        print(score_elem)
        if score_elem.text != '':
            scores.append(score_elem.text)
    print(player_name, scores)
    player_ele.click()


36
<selenium.webdriver.remote.webelement.WebElement (session="5e3a0f508c8ef9bd23bc34c75446cbfd", element="9edae0d5-de07-4d16-9f99-f00224d79c31")>
<selenium.webdriver.remote.webelement.WebElement (session="5e3a0f508c8ef9bd23bc34c75446cbfd", element="7f9512ed-0abe-483f-a78f-34e873cfc5fd")>
<selenium.webdriver.remote.webelement.WebElement (session="5e3a0f508c8ef9bd23bc34c75446cbfd", element="a46b48e6-6f4f-4e3a-ab3b-5f9d04b1644d")>
<selenium.webdriver.remote.webelement.WebElement (session="5e3a0f508c8ef9bd23bc34c75446cbfd", element="ed42253b-baa8-46a6-99bc-7994eae74ca6")>
<selenium.webdriver.remote.webelement.WebElement (session="5e3a0f508c8ef9bd23bc34c75446cbfd", element="84b12f84-574f-4d64-aee1-d8c9910aeb1b")>
<selenium.webdriver.remote.webelement.WebElement (session="5e3a0f508c8ef9bd23bc34c75446cbfd", element="6dd998bb-2ae1-42f2-9250-0c707bf17f9a")>
<selenium.webdriver.remote.webelement.WebElement (session="5e3a0f508c8ef9bd23bc34c75446cbfd", element="e4a8cd24-fa4f-4a36-8c69-81c1c6da5b0b

<selenium.webdriver.remote.webelement.WebElement (session="5e3a0f508c8ef9bd23bc34c75446cbfd", element="d7667635-fcfa-4962-994f-96c82ed6fda3")>
<selenium.webdriver.remote.webelement.WebElement (session="5e3a0f508c8ef9bd23bc34c75446cbfd", element="2a5201f9-45b1-41f0-9b60-70f151fc1cd7")>
<selenium.webdriver.remote.webelement.WebElement (session="5e3a0f508c8ef9bd23bc34c75446cbfd", element="ed370c8c-0a26-42e5-add1-642a8d12c485")>
<selenium.webdriver.remote.webelement.WebElement (session="5e3a0f508c8ef9bd23bc34c75446cbfd", element="84a1c290-3dd5-401c-9308-9bce5fdac7e2")>
<selenium.webdriver.remote.webelement.WebElement (session="5e3a0f508c8ef9bd23bc34c75446cbfd", element="ba1d2eec-4905-4afb-ba52-3c0b7788bf49")>
<selenium.webdriver.remote.webelement.WebElement (session="5e3a0f508c8ef9bd23bc34c75446cbfd", element="afe6acfd-0411-4ee3-8ee9-4e61035cf4cf")>
<selenium.webdriver.remote.webelement.WebElement (session="5e3a0f508c8ef9bd23bc34c75446cbfd", element="0c477c00-7ad6-4b7a-9d4c-4ecc7b653dce")>

KeyboardInterrupt: 

In [16]:
print(type(scores[0]))

<class 'selenium.webdriver.remote.webelement.WebElement'>
